# in this file i finde the best solotiion to my model.

## import lib need

In [41]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

## how make df
check distance

In [42]:
# current_dir = os.path.dirname(os.path.abspath(__file__))
# data_path = os.path.join(current_dir, "data_set_snake_game.csv")
df = pd.read_csv("data_set_snake_game.csv")
df = pd.get_dummies(df, columns=['direction'])

before_distance = int(np.sqrt(np.pow(df.loc[0]['food_x'] - df.loc[0]['head_x'],2) + np.pow(df.loc[0]['food_y'] - df.loc[0]['head_y'],2)))
for i in range(1,len(df)-1):
    current_distance = int(np.sqrt(np.pow(df.loc[i]['food_x'] - df.loc[i]['head_x'],2) + np.pow(df.loc[i]['food_y'] - df.loc[i]['head_y'],2)))
    if current_distance >= before_distance:
        # print("current_distance: %s, before_distance: %s, between distance: %s" %(current_distance, before_distance, before_distance-current_distance))
        # if current_distance - before_distance == 0:
            # print("head: ",df.loc[i]['head_x'], df.loc[i]['head_y'],"food: ", df.loc[i]['food_x'], df.loc[i]['food_y'], "distance: ", current_distance-before_distance)
            # print(np.pow(df.loc[i]['food_x'] - df.loc[i]['head_x'],2))
            # print(np.pow(df.loc[i]['food_y'] - df.loc[i]['head_y'],2))
            # print(np.pow(df.loc[i]['food_x'] - df.loc[i]['head_x'],2) + np.pow(df.loc[0]['food_y'] - df.loc[0]['head_y'],2))
            # print(np.sqrt(np.pow(df.loc[i]['food_x'] - df.loc[i]['head_x'],2) + np.pow(df.loc[i]['food_y'] - df.loc[i]['head_y'],2)))
            # print(int(np.sqrt(np.pow(df.loc[i]['food_x'] - df.loc[i]['head_x'],2) + np.pow(df.loc[i]['food_y'] - df.loc[i]['head_y'],2))))
            # print(before_distance)
            # print("*"*20)
        df = df.drop(index=i)
    before_distance = current_distance
df.reset_index()
df.head(10)

,head_x,head_y,food_x,food_y,body_len,distance_to_food,last_direction,direction_Down,direction_Left,direction_Right,direction_Up
0,100,150,50,225,4,90,Down,True,False,False,False
1,100,175,50,225,4,70,Down,True,False,False,False
2,100,200,50,225,4,55,Down,True,False,False,False
3,100,225,50,225,4,50,Down,True,False,False,False
12,250,200,50,225,4,201,Up,False,True,False,False
13,225,200,50,225,4,176,Left,False,True,False,False
14,200,200,50,225,4,152,Left,False,True,False,False
15,175,200,50,225,4,127,Left,False,True,False,False
16,150,200,50,225,4,103,Left,False,True,False,False
17,125,200,50,225,4,79,Left,False,True,False,False


## send data to torch

In [43]:
# x = df[['head_x' ,'head_y', 'food_x', 'food_y']].values.astype('float32')
# y = df[['direction_Down',	'direction_Left','direction_Right',	'direction_Up']].values

# print(x)

## Explanation of labels

In [44]:
df[['direction_Down',	'direction_Left','direction_Right',	'direction_Up']].sum()

direction_Down     170
direction_Left     247
direction_Right    247
direction_Up       158
dtype: int64

## new model with Decision Tree

In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# کدگذاری ستون last_direction به صورت عددی
direction_mapping = {"Down": 0, "Left": 1, "Right": 2, "Up": 3}
df["last_direction"] = df["last_direction"].map(direction_mapping)

# جدا کردن ویژگی‌ها (X) و خروجی‌ها (y)
X = df[["head_x", "head_y", "food_x", "food_y", "body_len", "distance_to_food", "last_direction"]].values
y = df[["direction_Down", "direction_Left", "direction_Right", "direction_Up"]].values

# تبدیل به Tensor
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)


class DecisionTree(nn.Module):
    def __init__(self):
        super(DecisionTree, self).__init__()
        # لایه کامل از 7 ویژگی ورودی به 4 کلاس خروجی
        self.fc = nn.Linear(7, 4)

    def forward(self, x):
        return self.fc(x)
def train_decision_tree(model, X, y, epochs=100, lr=0.01):
    # تعریف تابع هزینه
    loss_fn = nn.BCEWithLogitsLoss()  # مناسب برای خروجی چندکلاسه
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        # پیش‌بینی
        y_pred = model(X)
        
        # محاسبه خطا
        loss = loss_fn(y_pred, y)
        
        # به‌روزرسانی وزن‌ها
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

# ایجاد مدل
model = DecisionTree()

# آموزش مدل
train_decision_tree(model, X, y, epochs=100, lr=0.01)


def evaluate_decision_tree(model, X, y):
    with torch.no_grad():
        y_pred = torch.sigmoid(model(X))  # اعمال تابع سیگموید برای خروجی
        predicted_classes = (y_pred > 0.5).float()  # تبدیل خروجی به باینری (0 یا 1)
        accuracy = (predicted_classes == y).float().mean() * 100
        print(f"Accuracy: {accuracy:.2f}%")

# ارزیابی مدل
evaluate_decision_tree(model, X, y)



Epoch 10/100, Loss: 5.957223892211914
Epoch 20/100, Loss: 4.99464750289917
Epoch 30/100, Loss: 3.661952495574951
Epoch 40/100, Loss: 2.541675090789795
Epoch 50/100, Loss: 1.8115745782852173
Epoch 60/100, Loss: 1.3089302778244019
Epoch 70/100, Loss: 1.0198743343353271
Epoch 80/100, Loss: 0.8516382575035095
Epoch 90/100, Loss: 0.7727705836296082
Epoch 100/100, Loss: 0.6938928961753845
Accuracy: 79.77%


## nother

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split

# =========================
# مرحله ۱: آماده‌سازی داده‌ها
# =========================

# کدگذاری ستون last_direction به صورت عددی
direction_mapping = {"Down": 0, "Left": 1, "Right": 2, "Up": 3}
df["last_direction"] = df["last_direction"].map(direction_mapping)

# جدا کردن ویژگی‌ها (X) و خروجی‌ها (y)
X = df[["head_x", "head_y", "food_x", "food_y", "body_len", "distance_to_food", "last_direction"]].values
y = df[["direction_Down", "direction_Left", "direction_Right", "direction_Up"]].values

# نرمال‌سازی ویژگی‌های ورودی
X = X / X.max(axis=0)

# تقسیم داده‌ها به مجموعه‌های آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# تبدیل به Tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# =========================
# مرحله ۲: تعریف مدل
# =========================

class SnakeNet(nn.Module):
    def __init__(self):
        super(SnakeNet, self).__init__()
        self.fc1 = nn.Linear(7, 64)  # لایه ورودی به مخفی ۱
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)
        
        self.fc2 = nn.Linear(64, 128)  # مخفی ۱ به مخفی ۲
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        
        self.fc3 = nn.Linear(128, 64)  # مخفی ۲ به مخفی ۳
        self.relu3 = nn.ReLU()
        
        self.output = nn.Linear(64, 4)  # خروجی (۴ جهت)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu2(self.fc2(x))
        x = self.dropout2(x)
        x = self.relu3(self.fc3(x))
        x = self.output(x)
        return x

# مقداردهی اولیه وزن‌ها
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)

# =========================
# مرحله ۳: آموزش مدل
# =========================

def train_model(model, X_train, y_train, epochs=500, lr=1e-4):
    # تابع هزینه
    loss_fn = nn.BCEWithLogitsLoss()
    # بهینه‌ساز
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()

        # پیش‌بینی
        y_pred = model(X_train)

        # محاسبه خطا
        loss = loss_fn(y_pred, y_train)

        # به‌روزرسانی وزن‌ها
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

# =========================
# مرحله ۴: ارزیابی مدل
# =========================

def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        y_pred = torch.sigmoid(model(X_test))
        y_pred_classes = (y_pred > 0.5).float()
        accuracy = (y_pred_classes == y_test).float().mean()
        print(f"Accuracy: {accuracy * 100:.2f}%")

# =========================
# اجرای مدل
# =========================

# ایجاد مدل
model = SnakeNet()
model.apply(weights_init)  # مقداردهی اولیه وزن‌ها

# آموزش مدل
train_model(model, X_train, y_train, epochs=500, lr=1e-4)

# ارزیابی مدل
evaluate_model(model, X_test, y_test)


Epoch 50/500, Loss: nan
Epoch 100/500, Loss: nan
Epoch 150/500, Loss: nan
Epoch 200/500, Loss: nan
Epoch 250/500, Loss: nan
Epoch 300/500, Loss: nan
Epoch 350/500, Loss: nan
Epoch 400/500, Loss: nan
Epoch 450/500, Loss: nan
Epoch 500/500, Loss: nan
Accuracy: 75.00%
